# Курсовой проект по курсу "Python для DataScience часть 2"

## Импорт библиотек

In [ ]:
import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

from IPython.display import Markdown as md

import seaborn as sns

In [ ]:
test_path = 'course_project_test.csv'
train_path = 'course_project_train.csv'

## Построение модели классификации

### 1. Обзор обучающего датасета

In [ ]:
df_train = pd.read_csv(train_path)
df_train.head()

In [ ]:
df_train.shape

Видим, что имеется 7500 наблюдений и 17 признаков, из них один признак - Credit Default - целевой.

In [ ]:
df_train.iloc[0]

Описание датасета:

    Home Ownership - домовладение,
    Annual Income - годовой доход,
    Years in current job - количество лет на текущем месте работы,
    Tax Liens - налоговые обременения,
    Number of Open Accounts - количество открытых счетов,
    Years of Credit History - количество лет кредитной истории,
    Maximum Open Credit - наибольший открытый кредит,
    Number of Credit Problems - количество проблем с кредитом,
    Months since last delinquent - количество месяцев с последней просрочки платежа,
    Bankruptcies - банкротства,
    Purpose - цель кредита,
    Term - срок кредита,
    Current Loan Amount - текущая сумма кредита,
    Current Credit Balance - текущий кредитный баланс,
    Monthly Debt - ежемесячный долг,
    Credit Default - факт невыполнения кредитных обязательств (0 - погашен вовремя, 1 - просрочка)

In [ ]:
df_train.info()

Видим (там, где менее 7500), что есть пропуски. Это касается 'Annual Income', т.е. не у всех известен годовой доход. Что касается поля 'Month since last delinquent', то пропуски (NaN) можно заменить большим числом (фактически не было правонарушений). Таким числом может быть практически бесконечный срок кредита - 50 лет (600 месяцев).

In [ ]:
df_train.describe()

В целом, каких-то заоблачных значений нет, за исключением, возможно, суммы наибольшего открытого кредита кредита около 1300 млн. руб или текущей суммы кредита около 100 млн. руб. Из таблицы видно, что среднее значение 'Current Default' равно 0.28, что говорит о несбалансированности выборки (в данной выборке "дефолтников" около 28%).

### 2. Обработка выбросов

In [ ]:
col_name = 'Current Loan Amount'
ax = sns.boxplot(y = df_train[col_name])

Путем перебора выясняем, что подозрительным является значение "все девятки"

In [ ]:
val = 99999999
sum(df_train[col_name] == val)

Посмотрим на статистику данных в этом подозрительном разрезе

In [ ]:
df_train[df_train[col_name] == val][set(df_train.columns) - {col_name}].describe()

Видим, что это "недефолтники" (max и min 'Credit Default' равны 0), а так как недефолтников в выборке больше, то удаление части их не так страшно (тех, у которых подозрительное значение 'Current Loan Amount' 99999999).

Проверим признак 'Maximum Open Credit'

In [ ]:
col_name = 'Maximum Open Credit'
ax = sns.boxplot(y = df_train[col_name])

In [ ]:
val = 1304726170
sum(df_train[col_name] == val)

In [ ]:
df_train[df_train[col_name] == val][set(df_train.columns) - {col_name}]

Ничего особенного, но так как такое значение одно, то принимаем это за выброс и удаляем.

In [ ]:
df_train = df_train.drop(df_train[df_train[col_name] == val].index)

In [ ]:
val = 380052288
sum(df_train[col_name] == val)

In [ ]:
df_train[df_train[col_name] == val][set(df_train.columns) - {col_name}]

Ничего особенного. Опять удаляем выброс.

In [ ]:
df_train = df_train.drop(df_train[df_train[col_name] == val].index)

In [ ]:
val = 265512874
sum(df_train[col_name] == val)

In [ ]:
df_train[df_train[col_name] == val][set(df_train.columns) - {col_name}]

Ничего особенного.
Итого: удаляем наблюдения, у которых признак 'Current Loan Amount' равен 99999999.

In [ ]:
df_train = df_train.drop(df_train[df_train[col_name] == val].index)

In [ ]:
col_name = 'Current Loan Amount'
val = 99999999
df_train = df_train.drop(df_train[df_train[col_name] == val].index)

In [ ]:
df_train.describe()

Видим, что баланс выборки несколько улучшился (32% дефолтников), но ценой удаления 870 наблюдений (12%).

### 3. Обработка пропусков

Как уже говорилось ранее, заменим NaN в признаке 'Months since last delinquent' на 600 месяцев.

In [ ]:
col_name = 'Months since last delinquent'
val = 600
df_train[col_name].fillna(val, inplace = True)

In [ ]:
df_train.describe()

NaN в признаке 'Annual Income' заменим условной медианой (условной, т.е. зависящей от 'Credit Default').

In [ ]:
print(df_train[df_train['Credit Default'] == 0]['Annual Income'].median())
print(df_train[df_train['Credit Default'] == 1]['Annual Income'].median())

In [ ]:
col_name = 'Annual Income'
val_0 = df_train[df_train['Credit Default'] == 0][col_name].median()
val_1 = df_train[df_train['Credit Default'] == 0][col_name].median()
df_train.loc[df_train['Credit Default'] == 0, col_name] = df_train.loc[df_train['Credit Default'] == 0, col_name].fillna(val_0)
df_train.loc[df_train['Credit Default'] == 1, col_name] = df_train.loc[df_train['Credit Default'] == 1, col_name].fillna(val_1)

In [ ]:
df_train.describe()

Видим, что не обработан признак 'Bankruptcies'.

In [ ]:
col_name = 'Bankruptcies'
df_train[pd.isnull(df_train[col_name])][set(df_train.columns) - set(col_name)]

Удаляем эти наблюдения.

In [ ]:
df_train.dropna(subset = [col_name], inplace = True)

In [ ]:
df_train.describe()

Остался признак 'Credit Score'. Посмотрим на его значимость (корреляцию с другими, в т.ч с целевой переменной)

In [ ]:
corr_with_target = df_train.corr().iloc[:-1, -1].sort_values(ascending=False)

plt.figure(figsize=(12, 4))

sns.barplot(x=corr_with_target.values, y=corr_with_target.index)

plt.title('Correlation with target variable')
plt.show()

In [ ]:
col_name = 'Credit Default'
with_name = 'Credit Score'
df_tmp = df_train.copy()
df_tmp.dropna(subset = [with_name], inplace = True)
limit_bal_with_target_s = df_tmp[[with_name, col_name]]

sns.jointplot(x = col_name, y = with_name, data = limit_bal_with_target_s)
plt.show()

Видим, что 'Credit Score' в данном случае антирейтинг, т.е. если он большой (более 5000), то для данной выборки всегда следует дефолт. Если score менее 5000, то возможно два исхода, причем чаще нет дефолта.

In [ ]:
col_name = 'Credit Score'
threshold = 5000
print(sum((df_train[col_name] > threshold)))
print(sum((df_train[col_name] < threshold)))

Вероятность дефолта при score менее порога

In [ ]:
p1_down = sum((df_train[col_name] < threshold) & (df_train['Credit Default'] == 1)) / sum((df_train[col_name] < threshold))
print(format(p1_down, '.2f'))

In [ ]:
plt.figure(figsize=(12, 4))
sns.kdeplot(df_tmp.loc[(df_tmp['Credit Default'] == 0) & (df_train[col_name] < threshold), col_name], label = 'No Default')
sns.kdeplot(df_tmp.loc[(df_tmp['Credit Default'] == 1) & (df_train[col_name] < threshold), col_name], label = 'Default')

plt.title('Low Credit Score area')
plt.show()

Видим, что в нижней области рейтинга score его влияние правдоподобно: чем меньше в среднем рейтинг, тем выше вероятность дефолта (кривая 'Default' смещена левее кривой 'No Default'). Поэтому удалять рейтинг не стоит, а пропущенные значения стоит заменить на условную медиану.

In [ ]:
plt.figure(figsize=(12, 4))
sns.kdeplot(df_tmp.loc[(df_tmp['Credit Default'] == 0) & (df_train[col_name] > threshold), col_name], label = 'No Default')
sns.kdeplot(df_tmp.loc[(df_tmp['Credit Default'] == 1) & (df_train[col_name] > threshold), col_name], label = 'Default')

plt.title('High Credit Score area')
plt.show()

In [ ]:
col_name = 'Credit Score'
val_0 = df_train[df_train['Credit Default'] == 0][col_name].median()
val_1 = df_train[df_train['Credit Default'] == 0][col_name].median()
df_train.loc[df_train['Credit Default'] == 0, col_name] = df_train.loc[df_train['Credit Default'] == 0, col_name].fillna(val_0)
df_train.loc[df_train['Credit Default'] == 1, col_name] = df_train.loc[df_train['Credit Default'] == 1, col_name].fillna(val_1)

In [ ]:
df_train.describe()

### 4. Анализ данных

Выделим категориальные переменные.

In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
cat_col_names = list(df_train.select_dtypes(include = ['object']).columns)
print('Категориальные признаки')
print(cat_col_names)
tgt_col_names = ['Credit Default']
val_col_names = list(set(df_train.columns) - set(cat_col_names) - set(tgt_col_names))
print('Числовые признаки')
print(val_col_names)

Посмотрим какие значения могут принимать категориальные переменные

In [ ]:
for name in cat_col_names:
    print(name)
    print('*************')
    print(set(df_train[name]))

Видим значение NaN в признаке 'Years in current job'; видимо, это означает то, что клиент на данный момент не работает. Проанализируем корреляцию признаков с целевой переменной в разрезе категориальных определенных выше переменных.

In [ ]:
# Функция для построения корреляций признаков с целевой переменной в разрезах значений категориальной переменной col_name
def graph_corr(col_name, data, sz):
    cols = list(set(df_train[col_name]))
    cols = [str(c) for c in cols]
    for p in cols:
        eq = df_train[col_name].astype('str') == p
        count = sum(eq)
        corr_with_target = df_train[eq].corr().iloc[:-1, -1].sort_values(ascending = False)
        plt.figure(figsize = sz)
        sns.barplot(x = corr_with_target.values, y = corr_with_target.index)
        plt.title('Correlation with target variable. ' + col_name + ' = ' + str(p) + '. Count = ' + str(count))
        plt.xlim([-1, 1])
        plt.show()

- Домовладение

In [ ]:
col_name = 'Home Ownership'
graph_corr(col_name, df_train, (12,4))

- Количество лет на текущей работе

In [ ]:
col_name = 'Years in current job'
graph_corr(col_name, df_train, (12,4))

- Цель

In [ ]:
col_name = 'Purpose'
graph_corr(col_name, df_train, (12,4))

- Срочность

In [ ]:
col_name = 'Term'
graph_corr(col_name, df_train, (12,4))

В целом вывод таков, что разбиение на категории играет большую роль, и этим пренебрегать не следует, даже если наблюдений мало. 

### 5. Отбор признаков

Еще раз взглянем на корреляцию признаков с целевой переменной 'Credit Default'

In [ ]:
corr_with_target = df_train.corr().iloc[:-1, -1].sort_values(ascending=False)

plt.figure(figsize=(12, 4))

sns.barplot(x = corr_with_target.values, y = corr_with_target.index)

plt.title('Correlation with target variable')
plt.show()

Как обычно, не особенно информативно. Посмотрим на разные сочетания пар признаков.

In [ ]:
col_1 = 'Annual Income'
col_2 = 'Current Loan Amount'
show_kde = False
show_joint = True
plt.figure(figsize = (6, 6))
if show_kde:    
    sns.kdeplot(df_train[col_1], label = col_1)
    sns.kdeplot(df_train[col_2], label = col_2)

    plt.title(col_1 + ' vs ' + col_2)
    plt.show()

In [ ]:
tmp = df_train[[col_1, col_2]]

if show_joint:
    sns.jointplot(x = col_1, y = col_2, data = tmp)
    plt.show()

Видим линию, ограничивающую текущую сумму кредита сверху (upper limit), значит чем ниже годовой доход, тем в целом ниже текущая сумма кредита. Но, так как имеется ярко выраженный максимум частоты встречаемости годового дохода (т.е. яркая мода), то любопытно было бы взглянуть на условное (согласно моде) распределение текущей суммы кредита. Моду заменим на близкую медиану (величина непрерывная, поэтому моду трудно вычислять).

In [ ]:
tmp = df_train.loc[df_train[col_1] == df_train[col_1].median(), col_2]

sns.distplot(tmp)
plt.title('For most frequently ' + col_1 + ' value')
plt.show()

Поделим на "дефолтников" и "недефолтников"

In [ ]:
tmp_0 = df_train.loc[(df_train[col_1] == df_train[col_1].median()) & (df_train['Credit Default'] == 0), col_2]
tmp_1 = df_train.loc[(df_train[col_1] == df_train[col_1].median()) & (df_train['Credit Default'] == 1), col_2]

sns.distplot(tmp_0)
sns.distplot(tmp_1)
plt.title('For most frequently ' + col_1 + ' value')
plt.legend(['No Default', 'Default'])
plt.show()

В целом, у "дефолтников" текущая сумма кредита больше.

4. Анализ данных\n",
    "5. Отбор признаков\n",
    "6. Балансировка классов\n",
    "7. Подбор моделей, получение бейзлана\n",
    "8. Выбор наилучшей модели, настройка гиперпараметров\n",
    "9. Проверка качества, борьба с переобучением\n",
    "10. Интерпретация результатов\n",
    "\n",
    "**Прогнозирование на тестовом датасете**\n",
    "1. Выполнить для тестового датасета те же этапы обработки и постронияния признаков\n",
    "2. Спрогнозировать целевую переменную, используя модель, построенную на обучающем датасете\n",
    "3. Прогнозы должны быть для всех примеров из тестового датасета (для всех строк)\n",
    "4. Соблюдать исходный порядок примеров из тестового датасета"

## Прогнозирование на тестовом датасете